In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
import re
from sklearn.cluster import KMeans
import seaborn as sns

Contains a database of all daily returns of stocks in the S&P 500 index from 2017

In [2]:
df_panel_filtered = pd.read_csv("Data/S&P_data_s.csv")
df_panel_filtered.head()

,date,Ticker,Return
0,2017-01-03,ORCL,0.006502
1,2017-01-04,ORCL,0.004929
2,2017-01-05,ORCL,-0.002581
3,2017-01-06,ORCL,-0.004917
4,2017-01-09,ORCL,0.015085


In [3]:
df = df_panel_filtered.pivot_table(
    index='date',
    columns='Ticker',
    values=['Return']
)
df.columns = [f"{col_name}_{stock_name}" for col_name, stock_name in df]
df.fillna(0, inplace=True)
df.head()

,Return_A,Return_AABA,Return_AAL,Return_AAP,Return_AAPL,Return_AAXN,Return_ABBV,Return_ABC,Return_ABMD,Return_ABNB,...,Return_XRX,Return_XYL,Return_YELL,Return_YHOO,Return_YRCW,Return_YUM,Return_ZBH,Return_ZBRA,Return_ZION,Return_ZTS
date,,,,,,,,,,,,,,,,,,,,,
2017-01-03,0.020413,0.0,-0.008353,0.008751,0.002849,0.0,-0.003354,0.056529,-0.002840,0.0,...,0.103551,0.002625,0.0,0.005948,0.019578,-0.001895,0.001260,0.005714,0.003253,0.001121
2017-01-04,0.013121,0.0,0.008639,0.008206,-0.001119,0.0,0.014100,0.024815,0.030082,0.0,...,0.037736,0.014904,0.0,0.029820,0.044313,0.003639,0.009194,0.009043,0.014358,0.009703
2017-01-05,-0.011890,0.0,-0.017345,-0.000698,0.005085,0.0,0.007584,-0.011576,-0.008035,0.0,...,-0.004196,-0.009129,0.0,0.031952,-0.020509,0.003310,0.006425,-0.026198,-0.016210,-0.003327
2017-01-06,0.031156,0.0,0.006973,-0.013091,0.011148,0.0,0.000314,0.013384,0.005313,0.0,...,-0.015449,-0.007010,0.0,-0.002661,-0.013718,0.012097,0.000095,0.014277,0.006498,0.003152
2017-01-09,0.003126,0.0,0.018827,-0.000590,0.009160,0.0,0.006584,0.008019,0.014642,0.0,...,-0.005706,-0.004236,0.0,0.002668,-0.024890,0.002794,0.019436,0.000116,-0.010837,-0.002773


Performing rolling PCA (Principal Component Analysis) on the data and calculating the number of principal components needed to explain at least 90% of the variance

In [5]:
#rolling PCA
window_size = 60
explained_var_matrix = []

for start in range(len(df) - window_size + 1):
    window_data = df.iloc[start:start+window_size]
    pca = PCA()
    pca.fit(window_data)
    explained_var_matrix.append(pca.explained_variance_ratio_)

explained_var_matrix = np.array(explained_var_matrix)

avg_explained_var = np.mean(explained_var_matrix, axis=0)
cumulative_var = np.cumsum(avg_explained_var)

optimal_moving_components = np.argmax(cumulative_var >= 0.90) + 1
print(f"The minimum number of components that explains at least 90% of the variance: {optimal_moving_components}")

The minimum number of components that explains at least 90% of the variance: 36


In [ ]:
window_size = window_size
rolling_pca_components = []
rolling_dates = []

for start in range(len(df) - window_size + 1):
    end = start + window_size
    window_data = df.iloc[start:end]

    # (optimal_components)
    pca = PCA(n_components=optimal_moving_components)
    components = pca.fit_transform(window_data)

    last_component = components[-1]
    rolling_pca_components.append(last_component)
    rolling_dates.append(df.index[end - 1])

pca_columns = [f'PC{i+1}' for i in range(optimal_moving_components)]
rolling_pca_df = pd.DataFrame(rolling_pca_components, columns=pca_columns)
rolling_pca_df['date'] = rolling_dates
rolling_pca_df['date'] = pd.to_datetime(rolling_pca_df['date'])
rolling_pca_df.set_index('date', inplace=True)

print("PCA DataFrame כולל סטיית תקן:")
rolling_pca_df.head()

PCA DataFrame כולל סטיית תקן:


,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC27,PC28,PC29,PC30,PC31,PC32,PC33,PC34,PC35,PC36
date,,,,,,,,,,,,,,,,,,,,,
2017-03-29,0.118992,0.164933,-0.030665,0.000227,-0.081004,-0.002407,0.035930,-0.080298,-0.114003,0.034764,...,-0.028189,0.004309,-0.053378,-0.039055,-0.037664,-0.041278,0.045349,0.059483,0.085489,0.020776
2017-03-30,0.104784,-0.086174,-0.007083,-0.082395,0.104161,-0.017188,-0.085186,-0.101615,0.054778,-0.040017,...,-0.053863,0.032132,-0.089070,-0.058561,-0.097054,0.013656,-0.048404,-0.059154,-0.090315,-0.073443
2017-03-31,-0.002228,0.051638,-0.070500,0.054390,-0.011394,-0.001567,0.030182,0.019585,-0.031568,-0.009524,...,0.002692,0.027918,0.041773,0.017650,-0.026472,-0.001182,-0.023810,0.024872,0.025606,-0.032301
2017-04-03,-0.176949,-0.018420,0.051859,0.016895,0.002111,-0.006891,0.000454,-0.032912,-0.012422,0.032979,...,-0.033703,0.022163,0.027633,0.044635,0.008400,-0.091289,-0.040063,0.049228,0.012702,-0.019278
2017-04-04,-0.021850,-0.047530,0.232296,0.056476,0.014690,0.006374,0.040716,0.010891,-0.134485,0.048709,...,0.002217,-0.044126,-0.122925,-0.094595,-0.049931,-0.051987,0.064637,0.053295,-0.048040,0.027517
